## 8.4 Nullstellensuche im $\mathbb{R}^n$ 

In [ ]:
import numpy as np
from scripts.LR_Zerlegung import LR_zerlegung_mit_pivot, vorwaerts_einsetzen_ohne_diag, rueckwaerts_einsetzen

**Implementierung 8.5: Newton-Verfahren in  $\mathbb{R}^n$**

Um das normale Newton-Verfahren im Höher-Dimensionalen zu implementieren, müssen wir in jeder Iteration ein lineares Gleichungssystem lösen. Dazu verwenden wir unsere LR-Zerlegung mit Pivotisierung.

In [ ]:
def newton_vek(f, D, x, n=10, tol=1e-10):
    print('it  ||f(x)||')
    print('--------------')
    
    for i in range(n):
        b = - f(x)
        print(f'{i:02d} {np.linalg.norm(b): .4e}')
        if np.linalg.norm(b) < tol:
            return i, x
        
        jac = D(x)
        pivot = LR_zerlegung_mit_pivot(jac)
        
        for p in pivot:
            b[p] = b[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(jac, b)
        w = rueckwaerts_einsetzen(jac, y)
        x[:] += w
    else: 
        print(f'Das Newton-Verfahren ist nicht konvergiert')
        return i, x

#### Beispiel 8.32 (Newton im $\mathbb{R}^n$)

Wir suchen die Nullstelle der Funktion
$$f(x_1,x_2) = \begin{pmatrix}1-x_1^2-x_2^2 \\ (x_1-2x_2)/(1/2+x_2)\end{pmatrix}$$
mit der Jacobi-Matrix
$$ Df(x) = \begin{pmatrix} -2x_1 & -2x_2 \\ \frac{2}{1+2x_2} & -\frac{4+4x_1}{(1+2x_2)^2}\end{pmatrix}.$$
Die Nullstellen sind dabei
$$x \approx \pm (0.894427191, 0.447213595).$$
Für die Nullstellensuche implementieren wir zunächst die Funktion und Jacobi-Matrix.

In [ ]:
def f(x):
    x, y = x[:]
    return np.array([1 - x**2 - y**2, (x - 2 * y) / (1 / 2 + y)], dtype=np.double)

def Df(x):
    x, y = x[:]
    return np.array([[-2 * x, -2 * y],
                     [2 / (1 + 2 * y), - (4 + 4 * x)/ (1 + 2 * y)**2]], dtype=np.double)

Mit dem Startvektor $x_0 = (1, 1)^T$ erhalten wir dann die Lösung

In [ ]:
n, x = newton_vek(f, Df, x=np.array([1.0, 1.0]), n=15, tol=1e-10)

In [ ]:
print(f'x = {x} nach {n} Schritten')

und mit dem Startvektor $x_0 = (-1, -0.2)^T$ die zweite Nullstelle:

In [ ]:
n, x = newton_vek(f, Df, x=np.array([-1.0, -0.2]), n=15, tol=1e-10)
print(f'x = {x} nach {n} Schritten')

Der aufwändigste Schritt in dem Newton-Verfahren ist die Berechnung der LR-Zerlegung. Daher kann es sich im vektorwertigen Fall besonders lohnen das vereinfachte Newton-Verfahren zu verwenden.

**Implementierung 8.6: Vereinfachtes Newton-Verfahren in $\mathbb{R}^n$**

In [ ]:
def newton_vereinfacht_vek(f, D, x, n=10, tol=1e-10):
    jac = D(x)
    pivot = LR_zerlegung_mit_pivot(jac)
    # print(jac)
    
    for i in range(n):
        b = - f(x)
        if np.linalg.norm(b) < tol:
            return i, x
        for p in pivot:
            b[p] = b[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(jac, b)
        w = rueckwaerts_einsetzen(jac, y)
        x[:] += w
    else: 
        print(f'Das vereinfachte Newton-Verfahren ist nach {i} Iterationen konvergiert: {np.linalg.norm(f(x))}')
    return i, x

Wir wenden das vereinfachte Newton-Verfahren auf dasselbe Beispiel an. Dabei sehen wir, dass die Wahl der Startlösung (also die Stelle an der wir die Jacobi-Matrix invertieren) einen besonders großen Einfluss hat.

In [ ]:
n, x = newton_vereinfacht_vek(f, Df, x=np.array([1.0, 0.5]), n=50, tol=1e-10)

In [ ]:
print(f'x = {x} nach {n} Schritten')

In [ ]:
n, x = newton_vereinfacht_vek(f, Df, x=np.array([1.0, 1.0]), n=700, tol=1e-10)
print(f'x = {x} nach {n} Schritten')

In [ ]:
newton_vereinfacht_vek(f, Df, x=np.array([-1.0, 1.0]), n=1000, tol=1e-10)
print(f'x = {x} nach {n} Schritten')

Wie Sie sehen, kann es sein, dass das Verfahren nicht konvergiert.

Vergleichen Sie nun die Laufzeit des vereinfachten Newton-Verfahrens mit der jupyter cell-magic `%%timeit` mit $x_0 = (1, 0.5)$ mit dem Newton-Verfahren mit dem Startwert $x_0 = (1,1)$. Was beobachten Sie? Was schließen Sie dabei auf den Rechenaufwand der einzelnen Iterationsschritte der beiden Verfahren? Was erwarten Sie, wenn die Dimension des Problems wächst?

### 8.4.2 Globalisierung des Newton-Verfahrens

Wie wir gesehen haben, konvergiert das vereinfachte Newton-Verfahren nur manchmal. Es kann aber von Vorteil sein, da jeder einzelne Schritt deutlich schneller ist.

Unter Umständen kann das vektorwertige Newton-Verfahren sogar nur langsam oder gar nicht konvergieren.

In [ ]:
newton_vek(f, Df, x=np.array([0, -0.49999]), n=35, tol=1e-10)

Um den Konvergenzradius zu vergrößern, können wir die Schrittgröße des Newton-Verfahrens in jeder Iteration dämpfen.

In [ ]:
def newton_gedämpft_vek(f, D, x, omega, n=10, tol=1e-10):
    assert len(omega) == n, 'Anzahl Dämpfungsparameter und Schritte verschieden'

    print('it  ||f(x)||')
    print('--------------')
    
    for i in range(n):
        b = - f(x)
        print(f'{i:02d} {np.linalg.norm(b): .4e}')
        if np.linalg.norm(b) < tol:
            return i, x
        jac = D(x)
        pivot = LR_zerlegung_mit_pivot(jac)
        for p in pivot:
            b[p] = b[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(jac, b)
        w = rueckwaerts_einsetzen(jac, y)
        x[:] += omega[i] * w

    else: 
        print(f'Das gedämpfte Newton Verfahren ist nicht konvergiert')
        return i, x

#### Beispiel 8.35(Gedämpftes Newton-Verfahren) 

In [ ]:
newton_gedämpft_vek(f, Df, x=np.array([0, -0.49999]), omega=[0.88] * 50, n=50, tol=1e-10)

Sogar das vereinfachte Newton-Verfahren lässt sich unter Umständen durch eine geschickte Wahl der Dämpfung verbessern.

In [ ]:
def newton_vereinfacht_gedämpft_vek(f, D, x, omega, n=10, tol=1e-10):
    assert len(omega) == n, 'Anzahl Dämpfungsparameter und Schritte verschieden'
    jac = D(x)
    pivot = LR_zerlegung_mit_pivot(jac)
    for i in range(n):
        b = - f(x)
        if np.linalg.norm(b) < tol:
            return i, x
        for p in pivot:
            b[p] = b[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(jac, b)
        w = rueckwaerts_einsetzen(jac, y)
        x[:] += omega[i] * w
    else: 
        print(f'Das gedämpfte Newton Verfahren ist nicht konvergiert')
        return i, x

In [ ]:
newton_vereinfacht_gedämpft_vek(f, Df, x=np.array([1.0, 1.0]), omega=[0.74] * 30, n=30, tol=1e-10)

Es sind also nur noch 27 statt 670 Schritte jetzt notwendig

Da wir nun am Anfang nicht so weit in die falsche Richtung laufen, konvergiert das Verfahren schneller als das normale Newton-Verfahren. Allerdings erhalten wir nur langsame Konvergenz in jedem Schritt. Es ist also wichtig im Einzugsbereich der quadratischen Konvergenz auf das volle Newton-Verfahren zu wechseln. 

**Implementierung 8.7: Globalisiertes Newton-Verfahren**

Oben haben wir gesehen, dass das vereinfachte Newton-Verfahren 670 Schritte benötigt. Mit einer geschickten Wahl der Dämpfung konnten wir dies auf 27 reduzieren. Allerdings ist die korrekte Wahl von $\omega$ nicht einfach. Um im vektorwertigen vereinfachen Newton-Verfahren den Konvergenzbereich zu vergrößern, können wir hier auch eine Liniensuche einbauen um einen möglichst großen Dämpfungsparameter zu nehmen, und die Jacobi Matrix so zu aktualisieren, um die Konvergenz zu verbessern.

In [ ]:
def newton_global_vek(f, D, x, sigma=0.5, Lmax=10, n=10, tol=1e-10):
    b = - f(x)
    res0, res1 = np.linalg.norm(b), float('nan')
    if res0 < tol:
        return 0, x
    
    for i in range(n):
        if i == 0 or res0 / res1 > 0.3:
            print(f'i = {i}: Jacobi Matrix wird neu aufgestellt')
            jac = D(x)
            pivot = LR_zerlegung_mit_pivot(jac)
        for p in pivot:
            b[p] = b[[p[1], p[0]]]
        y = vorwaerts_einsetzen_ohne_diag(jac, b)
        w = rueckwaerts_einsetzen(jac, y)
        
        for l in range(Lmax):
            x_neu = x.copy() + sigma**l * w
            b = -f(x_neu)
            res2 = np.linalg.norm(b)
            if res2 < res0:
                if l > 0:
                    print(f'i = {i}: Es waren {l} Line-Search Schritte notwendig')
                x = x_neu
                break
        else:
            print(f'Schritt {i}: {l} Line-Search Schritte haben das Residuum nicht verbessert')
            x = x_neu
        
        res1 = res0
        res0 = res2
        if res0 < tol:
            return i, x
    else: 
        print(f'Die globalisierte Newton-Methode ist nicht konvergiert, res = {res0}')
        return i, x

Wenden wir dies nun an mit dem Startwert $x_0 = (1,1)^T$, wo bisher 670 Schritte notwendig waren, sehen wir

In [ ]:
n, x = newton_global_vek(f, Df, x=np.array([1.0, 1.0]), n=20, tol=1e-10)
print(f'\nn = {n}, x = {x}')

Durch eine Aktualisierung der Jacobi-Matrix, benötigen wir weniger als $1/40$ der Anzahl der Schritte im Vergleich zum vereinfachten Newton-Verfahren. Testen wir nun also noch den Startwert $x_0=(-1, 1)^T$, in dem das vereinfachte Newton-Verfahren nicht konvergiert ist

In [ ]:
n, x = newton_global_vek(f, Df, x=np.array([-1.0, 1.0]), n=50, tol=1e-10)
print(f'\nn = {n}, x = {x}')

Durch die Kombination der Line-Search und Aktualisierung der Jacobi-Matrix, konvergiert das Verfahren nun auch bei dieser Wahl des Startwertes.

Nehmen wir den Startwert $x=(0, -0.49999)$, bei dem die erste Jacobi-Matrix fast singulär ist, dann bekommen wir nun

In [ ]:
n, x = newton_global_vek(f, Df, x=np.array([0, -0.49999]), n=50, tol=1e-10)
print(f'\nn = {n}, x = {x}')

Also haben wir auch hier wieder Konvergenz erreicht. Es benötigt allerdings einige Schritte bevor wir in den Bereich kommen, wo eine Neuaufstellung der Jacobi-Matrix nicht mehr notwendig ist.